In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [3]:
activations = [e_swish_1, e_swish_2, swish, "relu", "sigmoid", "tanh"]
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_1', 'e_swish_2', 'swish', 'relu', 'sigmoid', 'tanh']


In [4]:
baseMapNum = 32
weight_decay = 1e-4

act, act_name = activations[1], names[1]


model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [5]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [6]:
#training
batch_size = 128
epochs=25
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75'+act_name+'.h5')

Epoch 1/75
390/390 [==============================] - 181s - loss: 2.0026 - acc: 0.4027 - val_loss: 1.2514 - val_acc: 0.5859
Epoch 2/75
390/390 [==============================] - 176s - loss: 1.3454 - acc: 0.5611 - val_loss: 1.0567 - val_acc: 0.6536
Epoch 3/75
390/390 [==============================] - 176s - loss: 1.1593 - acc: 0.6284 - val_loss: 0.9671 - val_acc: 0.6908
Epoch 4/75
390/390 [==============================] - 178s - loss: 1.0554 - acc: 0.6648 - val_loss: 0.8845 - val_acc: 0.7268
Epoch 5/75
390/390 [==============================] - 177s - loss: 0.9784 - acc: 0.6909 - val_loss: 0.8051 - val_acc: 0.7498
Epoch 6/75
390/390 [==============================] - 176s - loss: 0.9357 - acc: 0.7096 - val_loss: 0.7286 - val_acc: 0.7776
Epoch 7/75
390/390 [==============================] - 175s - loss: 0.8916 - acc: 0.7245 - val_loss: 0.7278 - val_acc: 0.7797
Epoch 8/75
390/390 [==============================] - 175s - loss: 0.8553 - acc: 0.7363 - val_loss: 0.7305 - val_acc: 0.7808


390/390 [==============================] - 173s - loss: 0.5586 - acc: 0.8518 - val_loss: 0.4937 - val_acc: 0.8801
Epoch 67/75
390/390 [==============================] - 173s - loss: 0.5629 - acc: 0.8524 - val_loss: 0.4980 - val_acc: 0.8792
Epoch 68/75
390/390 [==============================] - 173s - loss: 0.5575 - acc: 0.8537 - val_loss: 0.4805 - val_acc: 0.8825
Epoch 69/75
390/390 [==============================] - 173s - loss: 0.5630 - acc: 0.8523 - val_loss: 0.5168 - val_acc: 0.8714
Epoch 70/75
390/390 [==============================] - 173s - loss: 0.5601 - acc: 0.8539 - val_loss: 0.5028 - val_acc: 0.8732
Epoch 71/75
390/390 [==============================] - 173s - loss: 0.5602 - acc: 0.8540 - val_loss: 0.5084 - val_acc: 0.8738
Epoch 72/75
390/390 [==============================] - 173s - loss: 0.5545 - acc: 0.8536 - val_loss: 0.4763 - val_acc: 0.8866
Epoch 73/75
390/390 [==============================] - 173s - loss: 0.5563 - acc: 0.8535 - val_loss: 0.4910 - val_acc: 0.8794
Epoc

In [7]:
opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 175s - loss: 0.5211 - acc: 0.8667 - val_loss: 0.4567 - val_acc: 0.8931
Epoch 2/25
390/390 [==============================] - 174s - loss: 0.5072 - acc: 0.8693 - val_loss: 0.4436 - val_acc: 0.8944
Epoch 3/25
390/390 [==============================] - 174s - loss: 0.5010 - acc: 0.8718 - val_loss: 0.4408 - val_acc: 0.8969
Epoch 4/25
390/390 [==============================] - 174s - loss: 0.4922 - acc: 0.8735 - val_loss: 0.4411 - val_acc: 0.8982
Epoch 5/25
390/390 [==============================] - 174s - loss: 0.4918 - acc: 0.8740 - val_loss: 0.4431 - val_acc: 0.8934
Epoch 6/25
390/390 [==============================] - 174s - loss: 0.4858 - acc: 0.8737 - val_loss: 0.4335 - val_acc: 0.8986
Epoch 7/25
390/390 [==============================] - 174s - loss: 0.4844 - acc: 0.8746 - val_loss: 0.4353 - val_acc: 0.8974
Epoch 8/25
390/390 [==============================] - 174s - loss: 0.4801 - acc: 0.8765 - val_loss: 0.4349 - val_acc: 0.8960


In [8]:
opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 175s - loss: 0.4333 - acc: 0.8878 - val_loss: 0.4092 - val_acc: 0.9013
Epoch 2/25
390/390 [==============================] - 174s - loss: 0.4283 - acc: 0.8865 - val_loss: 0.4042 - val_acc: 0.9035
Epoch 3/25
390/390 [==============================] - 174s - loss: 0.4299 - acc: 0.8873 - val_loss: 0.3997 - val_acc: 0.9033
Epoch 4/25
390/390 [==============================] - 174s - loss: 0.4191 - acc: 0.8905 - val_loss: 0.4022 - val_acc: 0.9060
Epoch 5/25
390/390 [==============================] - 174s - loss: 0.4199 - acc: 0.8894 - val_loss: 0.4046 - val_acc: 0.9019
Epoch 6/25
390/390 [==============================] - 174s - loss: 0.4196 - acc: 0.8914 - val_loss: 0.3904 - val_acc: 0.9086
Epoch 7/25
390/390 [==============================] - 174s - loss: 0.4151 - acc: 0.8918 - val_loss: 0.3944 - val_acc: 0.9033
Epoch 8/25
390/390 [==============================] - 174s - loss: 0.4123 - acc: 0.8929 - val_loss: 0.3958 - val_acc: 0.9046


In [9]:
#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

 9984/10000 [============================>.] - ETA: 0s
Test result: 90.420 loss: 0.388
